In [1]:
#!/usr/bin/env python
# coding: utf-8

#i import the necessary libraries
import sys
import h5py
import numpy as np
import healpy as hp
import tensorflow as tf
import random as python_random
import nnhealpix.layers
from tensorflow.keras import metrics
import pandas as pd
from loss_functions import sigma_loss, sigma2_loss,sigma_batch_loss,sigma_norm_loss,sigma_log_loss,mse_tau,mse_sigma, mse_batch, sigma_f_loss
import math
import useful_functions as uf
import NN_functions as nuf
import os, shutil

seed_train=33
np.random.seed(seed_train)# i set a random seed for the generation of the maps for reproducibility

#map gen
nside = 16
n_train=100 #the total number of training+validation pair of maps that i will generate
n_train_fix=100 #the total number of of training maps i will spread on all the r interval -> for each r value i generate n_train_fix/len(r) maps 
kind_of_map="EE"
n_channels=2
pol=2
res=hp.nside2resol(nside, arcmin=False) 
sensitivity=4

name='2_6_23'
base_dir='/home/amorelli/QU_foreground_tau/'+name+'/'
# callbacks
reduce_lr_on_plateau = True
p_stopping=20
p_reduce=5
f_reduce=0.5
stopping_monitor="val_mse_batch"
reduce_monitor="val_loss"
metrics=[sigma_loss, sigma_batch_loss,mse_tau,mse_sigma, sigma_f_loss, mse_batch]# these are the different loss functions i have used. I use them as metrics

#network structure
one_layer=True # this is to switch between one dense layer or two dense layer
drop=0.2
n_layer_0=48
n_layer_1=64
n_layer_2=16
if kind_of_map!="QU": 
    n_inputs=n_channels
else:
    n_inputs=pol*n_channels

#train and val
batch_size = 16
max_epochs = 5
lr=0.0003 
fval=0.1 # this is the fraction of data that i use for validation, computed on n_train_fix
training_loss="new_sigma_batch"
loss_training=sigma_batch_loss#(0.01,True) # this is the loss i use for the training
shuffle=False

f_ = np.load('/home/amorelli/cl_generator/outfile_l_47_complete.npz') 
#print("outfile_R:",f_.files) #give the keiwords for the stored arrays
labels=f_.files
data=f_[labels[0]]
r=f_[labels[1]]
r, data=uf.unison_sorted_copies(r, data)
indexes=np.linspace(0,len(r)-1,5,dtype=int)
r=r[indexes]
data=data[indexes]

input_folder="/home/amorelli/foreground_noise_maps/noise_generation"
input_files=os.listdir(input_folder)
for i in range(len(input_files)):
    input_files[i]=input_folder+"/"+input_files[i]
noise_maps=uf.generate_noise_maps(n_train,n_channels,nside,pol=2,sensitivity=sensitivity,input_files=input_files)

noise_E,noise_B=uf.convert_to_EB(noise_maps)

maps_per_cl_gen=uf.maps_per_cl(distribution=0)
maps_per_cl=maps_per_cl_gen.compute_maps_per_cl(r,n_train,n_train_fix)

mappe_B,y_r=uf.generate_maps(data, r,n_train=n_train,nside=nside, map_per_cl=maps_per_cl, 
                             noise_maps=noise_E, beam_w=2*res, kind_of_map=kind_of_map, raw=0 , n_channels=n_channels,beam_yes=1 , verbose=0)

x_train,y_train,x_val,y_val = nuf.prepare_data(y_r,mappe_B,r,n_train,n_train_fix,fval,maps_per_cl
                                               , batch_size, batch_ordering=True)

#np.savez(base_dir+"check_r_distribution",y_train=y_train,y_val=y_val) 

#rand_indexes=np.random.randint(0,len(y_train)-1,10000)

#np.savez(base_dir+"check_train_maps",y_train=y_train[rand_indexes], x_train=x_train[rand_indexes])

model=nuf.build_network(n_inputs,nside,drop,n_layer_0,n_layer_1,n_layer_2,one_layer,
                        num_output=2,use_normalization=[False,True],use_drop=False,trainable=[True,True])

history=nuf.compile_and_fit(model, x_train, y_train, x_val, y_val, batch_size, max_epochs, 
                            stopping_monitor,p_stopping,reduce_monitor,f_reduce, p_reduce,base_dir, 
                            loss_training,lr,metrics,shuffle=shuffle, verbose=2,callbacks=[True,True,False,False])

print('Saving model to disk')
#model.save(base_dir+'test_model')
#-----------------------------------------
hyperparameters={}
hyperparameters["name"]=name
hyperparameters["loss"]=training_loss
hyperparameters["noise"]=sensitivity
hyperparameters["p_stopping"]=p_stopping
hyperparameters["p_reduce"]=p_reduce
hyperparameters["f_reduce"]=f_reduce
hyperparameters["stop-reduce"]=stopping_monitor+"-"+reduce_monitor
hyperparameters["lr"]=lr
hyperparameters["batch_size"]=batch_size
hyperparameters["n_layers"]=one_layer
if one_layer:
    hyperparameters["nodes_layers"]=n_layer_0
else:
    hyperparameters["nodes_layers"]=str(n_layer_1)+"-"+str(n_layer_2)
hyperparameters["comments"]=" "
hyperparameters = {k:[v] for k,v in hyperparameters.items()}
output=pd.DataFrame(hyperparameters)
output.to_csv(base_dir+'output.txt', index=False, sep=' ')

2023-06-10 18:31:57.370797: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-10 18:31:58.672934: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-10 18:31:58.673062: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-06-10 18:31:58.673079: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Epoch 1/5
5/5 - 5s - loss: 0.1180 - sigma_loss: 0.0687 - sigma_batch_loss: 0.1180 - mse_tau: 0.0667 - mse_sigma: 0.0019 - sigma_f_loss: 194371.5312 - mse_batch: 0.0513 - val_loss: 0.0229 - val_sigma_loss: 0.0282 - val_sigma_batch_loss: 0.0229 - val_mse_tau: 0.0189 - val_mse_sigma: 0.0093 - val_sigma_f_loss: 925679.6875 - val_mse_batch: 0.0040 - 5s/epoch - 962ms/step
Epoch 2/5
5/5 - 0s - loss: 0.0173 - sigma_loss: 0.0350 - sigma_batch_loss: 0.0173 - mse_tau: 0.0135 - mse_sigma: 0.0215 - sigma_f_loss: 2151180.5000 - mse_batch: 0.0038 - val_loss: 0.0169 - val_sigma_loss: 0.0261 - val_sigma_batch_loss: 0.0169 - val_mse_tau: 0.0014 - val_mse_sigma: 0.0247 - val_sigma_f_loss: 2466807.2500 - val_mse_batch: 0.0155 - 237ms/epoch - 47ms/step
Epoch 3/5
5/5 - 0s - loss: 0.0200 - sigma_loss: 0.0412 - sigma_batch_loss: 0.0200 - mse_tau: 0.0084 - mse_sigma: 0.0329 - sigma_f_loss: 3286584.5000 - mse_batch: 0.0116 - val_loss: 0.0115 - val_sigma_loss: 0.0232 - val_sigma_batch_loss: 0.0115 - val_mse_tau: